# Module 10: Variant Calling

## Overview

In this section, we will learn how to map sequence reads to a reference sequence and then identify the variants (both single nucleotide polymorphisms (SNPs) and small insertions and deletions (indels)) in the sequence data. We will then use the variants identified to generate a pseudogenome which is created by replacing the bases in reference genome by variants identified at corresponding positions.

### Install condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Install software

In [ ]:
# Install bwa
!conda install -c conda-forge -c bioconda -c defaults snippy

In [ ]:
# Install samtools
!conda config --add channels bioconda
!conda config --add channels conda-forge
!conda install -c bioconda samtools

### Download data

In [ ]:
!wget

## Part 1: Alignment

We will use the tool [Snippy](https://github.com/tseemann/snippy), which is an integrated pipeline that maps reads to a reference genome and finds variants between the reference genome and the sequence reads. It will use as many CPUs as you can give it on a single computer. It is designed with speed in mind, and produces a consistent set of output files in a single folder.

Snippy requires only three inputs: 

1. Reference genome in FASTA format
2. Sequence read file (s) in FASTQ format
3. A folder to put in the results

The following command is used to run snippy: 

In [ ]:
# Run snippy
!snippy --outdir GPSC46_folder --R1 17150_4#79_1.fastq.gz --R2 17150_4#79_2.fastq.gz --ref Reference_sequence_GPSC46.fa --cpus 2 --ram 2 --force --quiet

An explanation of this command is as follows:

`snippy`: es la herramienta

`--outdir`: specified the output directory which is GPSC46_folder

`--R1`: specifies the first read which is 17150_4#79_1.fastq.gz

`--R2`: specifies the second read which is 17150_4#79_2.fastq.gz

`--ref`: specifies the reference genome which is Reference_sequence_GPSC46.fa

`--cpus`: specifies maximum number of CPU cores to use = 2

`--ram`: specifies that the RAM is kept under 2 GB

`--force`: force overwrite of existing output folder

`--quiet`: no screen output

There should now be a new folder called GPSC46_folder. Now lets `cd` to this folder and list `ls -alh`the content of the directory to check we have our new files, and also check out their sizes. 

Deberá ver lo siguiente:

A description of the output files are as follows:

`ref.fa`: FASTA version/copy of the reference

`ref.ra.fai`: Index for the reference file

`snps.aligned.fa`: A version of the reference but with (-) at position with depth=0 and N for 0 < depth < --mincov (does not have variants)

`snps.bam`: The alignments in BAM format. Includes unmapped, multimapping reads. Excludes duplicates.

`snps.bam.bai`: Index for the .bam file

`snps.bed`: The variants in Browser Extensible Data (BED) format

`snps.consensus.fa`: A version of the reference genome with which contains all contains all high quality variants

`snps.consensus.subs.fa`: A version of the reference genome which contains only high quality SNPs (no INDELS)

`snps.tab`: A simple tab-separated summary of all the variants

`snps.csv`: A comma-separated version of the .tab file

`snps.txt`: Tab-separated columnar list of alignment/core-size statistics

`snps.filt.vcf`: The filtered variant calls from Freebayes

`snps.raw.vcf`: The unfiltered variant calls from Freebayes

`snps.vcf`: Multi-sample VCF file with genotype GT tags for all discovered alleles

`snps.vcf.gz`: Compressed .vcf file 

`snps.vcf.gz.csi`: Index for the .vcf.gz via bcftools index

`snps.gff`: The variants in GFF3 format

`snps.html`: A HTML version of the .tab file

`snps.log`: A log file with the commands run and their outputs

Snippy has done a number of things. You can examine the log file to see exactly what snippy has done. So type:

In [ ]:
!less snps.log

The output is as shown in the table below:

You can also use `grep` to retrieve the relevant lines for each command from the log file. 

In [ ]:
!grep outdir snps.log

You will get this output:

To see the bwa processing steps type the command:

In [ ]:
!grep bwa snps.log

You will get this output:

The command `bwa mem -Y -M -R '@RG\tID:GPSC46_folder\tSM:GPSC46_folder' -t 2 reference/ref.fa /data/17150_4#79_1.fastq.gz /data/17150_4#79_2.fastq.gz | samclip --max 10 --ref reference/ref.fa. fai | samtools sort -n -l 0 -T /tmp --threads 0 -m 1000M | samtools fixmate -m --threads 0 - - | samtools sort -l 0 -T /tmp --threads 0 -m 1000M | samtools markdup -T /tmp --threads 0 -r -s - - > snps.bam`   is a combination of several commands combined using pipes `|`. 

A description of this command is as follows:

`bwa mem -Y -M -R '@RG\tID:GPSC46_folder\tSM:GPSC46_folder' -t 2 reference/ref.fa /data/17150_4#79_1.fastq.gz /data/17150_4#79_2.fastq.gz`: This command uses bwa to map the sequence reads to the reference genome using bwa mem.

`samclip --max 10 --ref referencia/ref.fa.fai`: This command uses samtools to clip/remove  unaligned reads.

`samtools sort -n -l 0 -T /tmp --threads 0 -m 1000M`: This command tells samtools to fill in mate coordinates and insert size fields.

`samtools fixmate -m --threads 0 - -`: This command tells samtools to fill in mate coordinates and insert size fields.

`samtools sort -l 0 -T /tmp --threads 0 -m 1000M`: This command tells samtools to sort based on chromosome number and coordinates.

`samtools markdup -T /tmp --threads 0 -r -s - - `:This command uses samtools  to remove all the duplicates and also print some basic statistics about the results file. 

`>snps.bam`: snps.bam is the outputfile.

To see samtools processing type the command:

In [ ]:
!grep samtools snps.log

You will get this output

The command `bwa mem  -Y -M -R '@RG\tID:GPSC46_folder\tSM:GPSC46_folder' -t 2 reference/ref.fa /data/17150_4#79_1.fastq.gz /data/17150_4#79_2.fastq.gz | samclip --max 10 --ref reference/ref.fa.fai | samtools sort -n -l 0 -T /tmp --threads 0 -m 1000M | samtools fixmate -m --threads 0 - - | samtools sort -l 0 -T /tmp --threads 0 -m 1000M | samtools markdup -T /tmp --threads 0 -r -s - - > snps.bam`   has been explained above. 

To see freebayes processing type the command

In [ ]:
!grep freebayes snps.log

You will get this output

This command uses freebayes to identify potential variants with coordinates in variant call VCF format.

To see bcftools processing type the command

In [ ]:
!grep bcftools snps.log

You will get this output

The command `bcftools view --include 'FMT/GT="1/1" && QUAL>=100 && FMT/DP>=10 && (FMT/AO)/(FMT/DP)>=0' snps.raw.vcf | vt normalize-r reference/ref. fa - | bcftools annotate --remove '^INFO/TYPE,^INFO/DP,^INFO/RO,^INFO/AO,^INFO/AB,^FORMAT/GT,^FORMAT/DP,^FORMAT/RO,^FORMAT/AO,^FORMAT/QR,^FORMAT/QA,^FORMAT/GL' > snps.filt.vcf ` uses bcftools to filter the variants identified above using fixed thresholds.

The command `bcftools consensus --sample GPSC46_folder -f reference/ref.fa -o snps.consensus.fa snps.vcf.gz` uses bcftools to create a pseudogenome by replacing the bases in reference genome by the filtered substitutions(SNPs) identified above.

## Part 2: Viewing output

The standard output format for storing variant call information is VCF. [Here](https://samtools.github.io/hts-specs/VCFv4.2.pdf)  you can find more information for its interpretation.

To visualize the VCF, run the command:

In [ ]:
!head -n30 snps.vcf

You will get this output:

This command prints the first 30 lines of the snps.vcf file. 

The first 27 lines here are ‘headers’ and contain information about what has been done to call the variants, and helps you to interpret what different columns mean. All these lines begin with "#"

The last 3 lines are individual variants identified, one per line. Variants columns are labelled #CHROM POS ID REF ALT QUAL FILTER INFO FORMAT GPSC46_folder

We can view a slightly easier summary of these variants in the snps.tab file

In [ ]:
!head -n5 snps.tab

You will get this output:

This command prints the first 5 lines of the snps.vcf file. In this file, we have not provided gene information, so only the first 6 columns are relevant.

## Part 3: Pseudogenomes

Snippy has also created a fasta file with our pseudosequence genome which is made by replacing the reference bases with the snps identified.

In [ ]:
!head snps.consensus.subs.fa

You will get this output:

This command prints the first few lines of the file "snps.consensus.subs.fa".

## Part 4: Handling multiple genomes - creating multiple sequence alignment 

Running a set of isolate sequences (reads or contigs) against the same reference, you can use the snippy-multi script. 

Navigate to the multiple_files Links to an external site. folder and explore its contents. To execute the snippy-multi script, you will need:

- Paired end reads (single-end reads or assembled contigs)
- Tab separated input file (which contains a list of the forward and reverse reads in the following format: ID, names of R1 reads and names of R2 reads)
- Reference sequence

Explore the tab separated input file; it should have the following format

Now lets us use snippy-multi to generate a multiple sequence alignment file. 

First run this command to generate an output script:

In [ ]:
!snippy-multi list.txt --ref Reference_sequence_GPSC46.fa --cpus 2 > runme.sh

An explanation of this command is as follows:

`snippy-multi`: is the tool

`list.txt`: Tab separated input file (which contains a list of the forward and reverse reads in the following format: ID, names of R1 reads and names of R2 reads)

`--ref`: specifies the reference genome which is Reference_sequence_GPSC46.fa

`--cpus`: specifies maximum number of CPU cores to use = 2

`>runme.sh`: is the output script

You will get this output:

You can check that the script makes sense by running the command:

In [ ]:
!less runme.sh

You will get this output if the script is correct:

Now, generate the alignment by running the command:

In [ ]:
!sh ./runme.sh

An explanation of this command is as follows:

`sh`: is the command name of the Bourne shell, the standard command language interpreter of Unix and many Unix-like operating systems, including Linux

`/.runme.sh`: the output script in the current directory `/.`

>Note: the tool snippy-multi will also run snippy-core to generate the core genome SNP alignment files core.*  

Snippy has now created a number of files, including a ‘core SNP alignment’

Run this command to check output

In [ ]:
!ls -l core.*

An explanation of these commands is as follows:

`ls -l` is the command for long listing all items with core as prefix (core.*)

You will get this output

Viewing output

We have various files that summarise our variants, e.g.

Run this command to check the content of some these files:

In [ ]:
!head core.tab

You will get this output

And our multiple sequence alignment containing all genomes is in the file (core.tull.aln). View this file: 

In [ ]:
!head core.full.aln

You will get this output:

This file “core.full.aln” masks sequences with low confidence in different ways, but for some applications we want everything masked in the same way. Let’s change that so anything uncertain is marked as ’N’ using the snipp-clean_full_aln script that comes with snippy.

In [ ]:
!snippy-clean_full_aln core.full.aln > clean.full.aln

View this file: 

In [ ]:
!head clean.full.aln

You will get this output